# 💬 Stateful Chatbot with Gradio and Ollama

In this notebook, you'll explore how to build a **chatbot** that remembers your previous messages.

You'll learn how to:
- Use `gr.Chatbot` to display a back-and-forth conversation
- Maintain a **chat history** using `gr.State`
- Pass the full conversation history to a language model

---

### 🧠 How State Works

Each time you send a message, the chatbot:
1. Collects the full chat history (all previous user and model messages)
2. Sends it to the model, along with your new message
3. Adds the model's response to the history
4. Displays the updated conversation

You don’t need to fix anything in this example — just run the code and try talking to your friendly wizard! 🧙

In [ ]:
import gradio as gr
import ollama

MODEL = "gemma3:4b-it-qat"

# System prompt to set behavior
SYSTEM_PROMPT = "You are a helpful and kind wizard."

# Core function that takes the current message and chat history
def chat(user_input, history):
    if not user_input.strip():
        return "", history

    # Rebuild full message list including system prompt
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    for human, bot in history:
        messages.append({"role": "user", "content": human})
        messages.append({"role": "assistant", "content": bot})

    # Add the new user input
    messages.append({"role": "user", "content": user_input})

    # Call the model
    response = ollama.chat(
        model=MODEL,
        messages=messages
    )

    assistant_reply = response["message"]["content"]

    # Update chat history
    history.append((user_input, assistant_reply))

    return "", history, history

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🧙 Stateful LLM Chatbot\nTalk to a wizard who remembers your past messages.")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Your message", placeholder="Type something...")
    state = gr.State([])  # Stores the chat history

    msg.submit(chat, [msg, state], [msg, chatbot, state])

demo.launch(server_name="0.0.0.0", server_port=8080)
